In [3]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['표 설명', '표 요약']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    html_dict = {}
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            #### data preprocess
            if 'filename' in line:
                filename = line['filename']
                html = line['html']
                html_dict[filename] = html
                continue
            
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            if 'filename' in line:
                continue
            try:
                doc_title = line['table_meta']['table_meta.doc_title']
                table_title = line['table_meta']['table_meta.table_title']
                html_path = line['table_meta']['table_meta.html_path'].split('/')[-1]
                html = html_dict[html_path]
                
                explanation = line['table_data']['table_data.text_explanation']
                summary = line['table_data']['table_data.text_summary']
            except:
                continue
            
            #### data preprocess end 
            
            #### 표 설명
            data = {'input': {}, 'output': None}
            ## preprocess data from line
            data['input']['doc_title'] = doc_title
            data['input']['table_title'] = table_title
            data['input']['html'] = html
            data['output'] = explanation
            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 표 설명 end
            
            
            #### 표 요약
            data = {'input': {}, 'output': None}
            ## preprocess data from line
            data['input']['doc_title'] = doc_title
            data['input']['table_title'] = table_title
            data['input']['html'] = html
            data['output'] = summary
            ## preprocess data from line end
            task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 표 요약 end
            
            
            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 48/48 [00:02<00:00, 21.35it/s]
